## types and mulitple dispatch

In [2]:
typeof("foo")

String

In [3]:
typeof('c')

Char

In [4]:
typeof(0.5)

Float64

For example, on an abstract mathematical level we don’t distinguish between 1 + 1 and 1.0 + 1.0

But for a CPU, integer and floating point addition are different things, using a different set of instructions

Julia handles this problem by storing multiple, specialized versions of functions like addition, one for each data type or set of data types

These individual specialized versions are called methods

When an operation like addition is requested, the Julia runtime environment inspects the type of data to be acted on and hands it out to the appropriate method

This process is called multiple dispatch

In [5]:
+(1, 1)

2

In [6]:
x, y = 1.0, 1.0
@which +(x, y)

+(x::Float64, y::Float64) in Base at float.jl:375

In [7]:
x, y = 1, 1
@which +(x, y)

+(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:32

In [8]:
x, y = 1.0 + 1.0im, 1.0 + 1.0im
@which +(x, y)

+(z::Complex, w::Complex) in Base at complex.jl:221

In [9]:
isless(1.0, 2.0)  # Applied to two floats

true

In [10]:
@which isless(1.0, 2.0)

isless(x::Float64, y::Float64) in Base at float.jl:439

In [11]:
@which isless(1, 2)

isless(x::Real, y::Real) in Base at operators.jl:266

In [12]:
@which isfinite(1) # Call isfinite on an integer

isfinite(x::Integer) in Base at float.jl:541

In [13]:
@which isfinite(1.0) # Call isfinite on a float

isfinite(x::AbstractFloat) in Base at float.jl:538

In [14]:
methods(isfinite)

# 9 methods for generic function "isfinite":
isfinite(x::BigFloat) in Base.MPFR at mpfr.jl:831
isfinite(x::Float16) in Base at float.jl:539
isfinite(x::AbstractFloat) in Base at float.jl:538
isfinite(x::Integer) in Base at float.jl:541
isfinite(::Irrational) in Base at irrationals.jl:102
isfinite(x::Real) in Base at float.jl:540
isfinite(z::Complex) in Base at complex.jl:117
isfinite(::Union{T, Type{T}}) where T<:Base.Dates.TimeType in Base.Dates at dates\types.jl:316
isfinite(x::AbstractArray{T,N} where N) where T<:Number in Base at deprecated.jl:56

In [15]:
+(100, "100")

LoadError: [91mMethodError: no method matching +(::Int64, ::String)[0m
Closest candidates are:
  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424
  +(::T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}, [91m::T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}[39m) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at int.jl:32
  +(::Integer, [91m::Ptr[39m) at pointer.jl:128
  ...[39m

In [17]:
#can change the sensible behaviour
importall Base.Operators  #  Gives access to + so that we can add a method

+(x::Integer, y::String) = x + parse(Int, y)

+ (generic function with 181 methods)

In [18]:
+(100, "100")

200

In [19]:
100 + "100"

200

## Dispatch and User-Defined Functions

In [20]:
function h(a::Float64)
    println("You have called the method for handling Float64s")
end

function h(a::Int64)
    println("You have called the method for handling Int64s")
end

h (generic function with 2 methods)

In [21]:
h(1.0)

You have called the method for handling Float64s


In [22]:
h(1)

You have called the method for handling Int64s


## The Type Hierarchy

The answer is that, in the Julia language specification, the types form a hierarchy

For example, Float64 and Int64 are subtypes of Real

In [23]:
Float64 <: Real

true

In [24]:
Complex64 <: Number

true

Number in turn is a subtype of Any, which is a parent of all types

In [25]:
supertype(Float64)

AbstractFloat

In [26]:
supertype(Complex64)

Number

In [27]:
+(100, "100")

200

In [28]:
function f(x)
    println("Generic function invoked")
end

function f(x::Number)
    println("Number method invoked")
end

function f(x::Integer)
    println("Integer method invoked")
end

f (generic function with 3 methods)

In [29]:
f(3)

Integer method invoked


In [30]:
f(3.0)

Number method invoked


In [31]:
f("foo")

Generic function invoked


## User-Defined Types

Syntax
While there are multiple ways to create new types, we almost always use the struct keyword, which is for creation of composite data types

Notes:

“composite” refers to the fact that the data types in question can be used as “containers” that hold a variety of data
the struct terminology is used in a number of programming languages to refer to composite data types

In [32]:
struct Foo  # A useless data type that stores no data
end

In [33]:
foo = Foo()  # Call default constructor, make a new Foo

Foo()

In [34]:
typeof(foo)

Foo

## Adding Methods

In [35]:
foofunc(x::Foo) = "onefoo"

foofunc (generic function with 1 method)

In [36]:
foofunc(foo)

"onefoo"

In [37]:
+(x::Foo, y::Foo) = "twofoos"

+ (generic function with 182 methods)

In [39]:
foo1, foo2 = Foo(), Foo()

(Foo(), Foo())

In [40]:
+(foo1, foo2)

"twofoos"

### A Less Trivial Example

In [42]:
mutable struct AR1
    a
    b
    σ
    ϕ
end

In [43]:
using Distributions

In [44]:
m = AR1(0.9, 1, 1, Beta(5, 5))

AR1(0.9, 1, 1, Distributions.Beta{Float64}(α=5.0, β=5.0))

In this call to the constructor we’ve created an instance of AR1 and bound the name m to it

We can access the fields of m using their names and “dotted attribute” notation

In [45]:
m.a

0.9

In [46]:
m.b

1

In [47]:
m.σ

1

In [48]:
m.ϕ

Distributions.Beta{Float64}(α=5.0, β=5.0)

In [49]:
typeof(m.ϕ)

Distributions.Beta{Float64}

In [50]:
typeof(m.ϕ) <: Distribution

true

In [51]:
m.ϕ = Exponential(0.5)

Distributions.Exponential{Float64}(θ=0.5)

In our type definition we can be explicit that we want ϕ to be a Distribution and the other elements to be floats

In [52]:
struct AR1_explicit
    a::Float64
    b::Float64
    σ::Float64
    ϕ::Distribution
end

In [53]:
m = AR1_explicit(0.9, 1, "foo", Beta(5, 5))

LoadError: [91mMethodError: Cannot `convert` an object of type String to an object of type Float64
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.[39m

In [54]:
struct AR1_real
    a::Real
    b::Real
    σ::Real
    ϕ::Distribution
end

## Type Parameters
Fortunately, there’s another approach that both

preserves the use of concrete types for internal data and
allows flexibility across multiple concrete data types

In [55]:
typeof([10, 20, 30])

Array{Int64,1}

Here Array is one of Julia’s predefined types (Array <: DenseArray <: AbstractArray <: Any)

The Int64,1 in curly brackets are type parameters

In this case they are the element type and the dimension

In [56]:
struct AR1_best{T <: Real}
    a::T
    b::T
    σ::T
    ϕ::Distribution
end

For the coefficients a, b and σ we considered

allowing them to be any type
forcing them to be of type Float64
allowing them to be any Real , which is chosen here

In [57]:
m = AR1_best(0.9, 1.0, 1.0, Beta(5, 5))

AR1_best{Float64}(0.9, 1.0, 1.0, Distributions.Beta{Float64}(α=5.0, β=5.0))

Exercise 1

Write a function with the signature simulate(m::AR1, n::Integer, x0::Real) that takes as arguments

an instance m of AR1 (see above)
an integer n

a real number x0 

and returns an array containing a time series of length n generated according to (1) where

the primitives of the AR(1) process are as specified in m

the initial condition X0 is set equal to x0

Hint: If d is an instance of Distribution then rand(d) generates one random draw from the distribution specified in d

In [58]:
struct AR1_ex1{T <: Real}
    a::T
    b::T
    σ::T
    ϕ::Distribution
end

In [59]:
function simulate(m::AR1_ex1, n::Integer, x0::Real)
    X = Array{Float64}(n)
    X[1] = x0
    for t in 1:(n-1)
        X[t+1] = m.a * X[t] + m.b + m.σ * rand(m.ϕ)
    end
    return X
end

simulate (generic function with 1 method)

In [60]:
m = AR1_ex1(0.9, 1.0, 1.0, Beta(5, 5))
X = simulate(m, 100, 0.0)

100-element Array{Float64,1}:
  0.0    
  1.50821
  2.84274
  4.0285 
  5.21339
  6.05818
  7.1221 
  7.85769
  8.2897 
  8.94837
  9.37839
  9.84273
 10.5033 
  ⋮      
 15.6966 
 15.6537 
 15.7133 
 15.5215 
 15.5588 
 15.6726 
 15.7077 
 15.4621 
 15.7521 
 15.8148 
 15.7466 
 15.7259 

INFO: Precompiling module Plots.
This may mean module Compat does not support precompilation but is imported by a module that does.
ERROR: LoadError: Declaring __precompile__(false) is not allowed in files that are being precompiled.
Stacktrace:
 [1] _require(::Symbol) at .\loading.jl:455
 [2] require(::Symbol) at .\loading.jl:405
 [3] _include_from_serialized(::String) at .\loading.jl:157
 [4] _require_from_serialized(::Int64, ::Symbol, ::String, ::Bool) at .\loading.jl:200
 [5] _require_search_from_serialized(::Int64, ::Symbol, ::String, ::Bool) at .\loading.jl:236
 [6] _require(::Symbol) at .\loading.jl:441
 [7] require(::Symbol) at .\loading.jl:405
 [8] include_from_node1(::String) at .\loading.jl:576
 [9] include(::String) at .\sysimg.jl:14
 [10] anonymous at .\<missing>:2
while loading C:\Users\z5187692\AppData\Local\JuliaPro-0.6.2.2\pkgs-0.6.2.2\v0.6\Plots\src\Plots.jl, in expression starting on line 7


LoadError: [91mFailed to precompile Plots to C:\Users\z5187692\AppData\Local\JuliaPro-0.6.2.2\pkgs-0.6.2.2\lib\v0.6\Plots.ji.[39m